In [1]:
import sys
import os
sys.path.append(os.path.abspath("../"))  # or "../../" depending on location

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.nn.functional as F
from utils.load_models import set_device, get_CNN_small, get_CNN, get_resnet
import numpy as np
from tqdm import tqdm

In [3]:
# --- パラメータ設定 ---
batch_size = 128
epsilon = 1  # 摂動の最大幅
alpha = 0.01   # 学習率
num_epochs = 5
patch_size = (8, 8)  # 左上に貼るパッチのサイズ
device = set_device()

In [4]:

# ==========================
# 変換定義（正規化なし）
# ==========================
transform = transforms.ToTensor()
normalize = transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                 std=(0.2023, 0.1994, 0.2010))

# ==========================
# CIFAR-10データ読み込み
# ==========================

# ダウンロード済みのデータを指定して読み込み
root = "../data/"
train_dataset = datasets.CIFAR10(
    root=root, 
    train=True, 
    download=False,  
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
subset_size = 10000
indices = np.random.choice(len(train_dataset), subset_size, replace=False)

# サブセットデータセットを作成
small_train_dataset = Subset(train_dataset, indices)
train_loader_small = DataLoader(small_train_dataset, batch_size=128, shuffle=True)


# --- パッチのマスク生成関数 ---
def generate_mask(size=(8, 8), image_shape=(3, 32, 32)):
    mask = torch.zeros(image_shape)
    mask[:, 0:size[0], 0:size[1]] = 1.0
    return mask

# --- マスクとパッチ初期化 ---
mask = generate_mask(size=patch_size).to(device)
mask = mask.unsqueeze(0)  # [1, 3, 32, 32]
delta = torch.zeros((1, 3, 32, 32), requires_grad=True, device=device)

def train_local_uap(
    model, 
    save_path,
    train_dataset=None,
    image_shape=(3, 32, 32),
    patch_size=(8, 8),
    epsilon=0.1,
    alpha=0.01,
    batch_size=128,
    num_epochs=5,
    device=None
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    model.eval()

    # --- データセットの準備（未指定ならCIFAR10を使う） ---
    if train_dataset is None:
        transform = transforms.ToTensor()
        train_dataset = datasets.CIFAR10(
            root="../data", train=True, download=True, transform=transform
        )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # --- マスクと摂動の初期化 ---
    mask = generate_mask(patch_size, image_shape).to(device).unsqueeze(0)  # [1, C, H, W]
    delta = torch.zeros((1, *image_shape), requires_grad=True, device=device)

    for epoch in range(num_epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            adv_images = (images + delta * mask).clamp(0, 1)
            outputs = model(adv_images)
            loss = F.cross_entropy(outputs, labels)

            loss.backward()
            with torch.no_grad():
                grad = delta.grad
                delta += alpha * torch.sign(grad) * mask
                delta.clamp_(-epsilon, epsilon)
                delta.grad.zero_()
            total_loss += loss.item()

        print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}")

    # --- 保存 ---
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(delta.detach().cpu(), save_path)
    print(f"Local UAP saved to {save_path}")

    return delta.detach()
def train_local_uap_with_confidence_penalty(
    model,
    save_path,
    train_dataset=None,
    image_shape=(3, 32, 32),
    patch_size=(8, 8),
    epsilon=0.1,
    alpha=0.01,
    batch_size=128,
    num_epochs=5,
    lambda_conf=0.5,
    device=None,
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    model.eval()

    # --- データセット準備 ---
    if train_dataset is None:
        transform = transforms.ToTensor()
        train_dataset = datasets.CIFAR10(
            root="../data", train=True, download=True, transform=transform
        )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # --- マスクと摂動の初期化 ---
    def generate_mask(patch_size=(8, 8), image_shape=(3, 32, 32)):
        mask = torch.zeros(image_shape)
        h, w = patch_size
        mask[:, :h, :w] = 1.0
        return mask.unsqueeze(0).to(device)

    mask = generate_mask(patch_size, image_shape)
    delta = torch.zeros((1, *image_shape), requires_grad=True, device=device)

    # --- 学習ループ ---
    for epoch in range(num_epochs):
        total_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)
            adv_images = (images + delta * mask).clamp(0, 1)
            outputs = model(adv_images)

            ce_loss = F.cross_entropy(outputs, labels)
            probs = F.softmax(outputs, dim=1)
            correct_class_probs = probs[range(len(labels)), labels]
            conf_penalty = correct_class_probs.mean()

            loss = ce_loss - lambda_conf * conf_penalty  # ← 修正済み符号

            loss.backward()
            with torch.no_grad():
                grad = delta.grad
                delta += alpha * torch.sign(grad) * mask
                delta.clamp_(-epsilon, epsilon)
                delta.grad.zero_()
            total_loss += loss.item()

        print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}")

    # --- 保存 ---
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(delta.detach().cpu(), save_path)
    print(f"Local UAP (with confidence penalty) saved to {save_path}")

    return delta.detach()

def train_local_uap_with_entropy_maximization(
    model,
    save_path,
    train_dataset=None,
    image_shape=(3, 32, 32),
    patch_size=(8, 8),
    epsilon=0.1,
    alpha=0.01,
    batch_size=128,
    num_epochs=5,
    lambda_entropy=1.0,
    device=None,
):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    model.eval()

    # --- データセットの準備（未指定ならCIFAR10） ---
    if train_dataset is None:
        transform = transforms.ToTensor()
        train_dataset = datasets.CIFAR10(
            root="../data", train=True, download=True, transform=transform
        )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # --- マスクと摂動の初期化 ---
    def generate_mask(patch_size=(8, 8), image_shape=(3, 32, 32)):
        mask = torch.zeros(image_shape)
        h, w = patch_size
        mask[:, :h, :w] = 1.0
        return mask.unsqueeze(0).to(device)

    mask = generate_mask(patch_size, image_shape)
    delta = torch.zeros((1, *image_shape), requires_grad=True, device=device)

    # --- 学習ループ ---
    for epoch in range(num_epochs):
        total_loss = 0.0
        for images, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images = images.to(device)
            adv_images = (images + delta * mask).clamp(0, 1)
            outputs = model(adv_images)

            # --- softmax entropy 最大化 ---
            probs = F.softmax(outputs, dim=1)
            entropy = -(probs * probs.log()).sum(dim=1).mean()

            # --- 損失（最大化したいのでそのまま） ---
            loss = lambda_entropy * entropy

            loss.backward()
            with torch.no_grad():
                grad = delta.grad
                delta += alpha * torch.sign(grad) * mask
                delta.clamp_(-epsilon, epsilon)
                delta.grad.zero_()
            total_loss += loss.item()

        print(f"[Epoch {epoch+1}] Entropy Loss: {total_loss:.4f}")

    # --- 保存 ---
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(delta.detach().cpu(), save_path)
    print(f"Entropy-maximizing Local UAP saved to {save_path}")

    return delta.detach()

In [5]:
cnn = get_CNN()
cnn = cnn.to(device)
cnn_local_uap_path = "local_uap/local_uap_delta_CNN.pt"
train_local_uap(cnn,cnn_local_uap_path,train_dataset=train_dataset,epsilon=epsilon,device=device)

[Epoch 1] Loss: 3833.5911
[Epoch 2] Loss: 4296.2182
[Epoch 3] Loss: 4297.0557
[Epoch 4] Loss: 4296.8686
[Epoch 5] Loss: 4297.0945
Local UAP saved to local_uap/local_uap_delta_CNN.pt


tensor([[[[-0.3300, -0.9400, -0.0500,  ...,  0.0000,  0.0000,  0.0000],
          [-1.0000, -0.7200,  0.7900,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.8500, -0.2100, -0.9100,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000, -1.0000, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.1700, -0.8500,  0.7300,  ...,  0.0000,  0.0000,  0.0000],
          [-0.1500, -0.1100, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.4200, -0.5700, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.4300, -0.9900, -

In [6]:
cnn_small = get_CNN_small()
cnn_small = cnn_small.to(device)
cnn_small_local_uap_path = "local_uap/local_uap_delta_CNN_small.pt"
train_local_uap(cnn_small,cnn_small_local_uap_path,train_dataset=small_train_dataset,epsilon=epsilon,device=device)

[Epoch 1] Loss: 396.1982
[Epoch 2] Loss: 436.4722
[Epoch 3] Loss: 438.8526
[Epoch 4] Loss: 439.4145
[Epoch 5] Loss: 440.0472
Local UAP saved to local_uap/local_uap_delta_CNN_small.pt


tensor([[[[-1.0000, -0.8100, -0.9600,  ...,  0.0000,  0.0000,  0.0000],
          [-0.2300,  0.8700,  0.0900,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  0.2500,  0.4500,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000, -1.0000,  0.2700,  ...,  0.0000,  0.0000,  0.0000],
          [-0.9900, -0.3900, -0.9200,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0700, -0.9700, -0.2500,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000,  1.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  0.9900,  

In [7]:
cnn = get_CNN()
cnn = cnn.to(device)
cnn_local_uap_path_withConfidencePenalty = "local_uap/local_uap_delta_CNN_withConfidencePenalty.pt"
train_local_uap_with_confidence_penalty(
    cnn,
    cnn_local_uap_path_withConfidencePenalty,
    train_dataset=train_dataset,
    epsilon=epsilon,
    device=device
)


Epoch 1/5: 100%|██████████| 391/391 [00:04<00:00, 92.89it/s]


[Epoch 1] Loss: 3362.3385


Epoch 2/5: 100%|██████████| 391/391 [00:04<00:00, 88.89it/s] 


[Epoch 2] Loss: 3769.6141


Epoch 3/5: 100%|██████████| 391/391 [00:03<00:00, 100.52it/s]


...

Epoch 4/5: 100%|██████████| 391/391 [00:03<00:00, 99.80it/s] 


[Epoch 4] Loss: 3775.1000


Epoch 5/5: 100%|██████████| 391/391 [00:05<00:00, 77.20it/s]

[Epoch 5] Loss: 3775.1428
Local UAP (with confidence penalty) saved to local_uap/local_uap_delta_CNN_withConfidencePenalty.pt


tensor([[[[-1.0000, -0.0100,  0.0700,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0900, -0.3900, -0.8900,  ...,  0.0000,  0.0000,  0.0000],
          [-0.3100,  1.0000, -0.4100,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-0.4100, -0.9900,  0.6900,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.0000, -0.6000, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.3700,  1.0000, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-1.0000, -0.6300, -0.8300,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.8300, -1.0000, -

In [8]:
cnn_small = get_CNN_small()
cnn_small = cnn_small.to(device)
cnn_small_local_uap_path_withConfidencePenalty = "local_uap/local_uap_delta_CNN_small_withConfidencePenalty.pt"
train_local_uap_with_confidence_penalty(
    cnn_small,
    cnn_small_local_uap_path_withConfidencePenalty,
    train_dataset=small_train_dataset,
    epsilon=epsilon,
    device=device
)



Epoch 1/5: 100%|██████████| 79/79 [00:01<00:00, 43.22it/s]


[Epoch 1] Loss: 389.5329


Epoch 2/5: 100%|██████████| 79/79 [00:00<00:00, 82.37it/s]


[Epoch 2] Loss: 429.4489


Epoch 3/5: 100%|██████████| 79/79 [00:00<00:00, 91.86it/s]


...

Epoch 4/5: 100%|██████████| 79/79 [00:00<00:00, 103.25it/s]


[Epoch 4] Loss: 435.3892


Epoch 5/5: 100%|██████████| 79/79 [00:01<00:00, 72.77it/s] 

[Epoch 5] Loss: 435.2501
Local UAP (with confidence penalty) saved to local_uap/local_uap_delta_CNN_small_withConfidencePenalty.pt


tensor([[[[-0.6500, -1.0000, -0.9900,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.3300,  0.9300,  0.1300,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.9900,  0.0500,  0.3700,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000, -1.0000,  0.5500,  ...,  0.0000,  0.0000,  0.0000],
          [-0.8300, -0.1300, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0100, -0.8700,  0.1900,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 1.0000,  0.4100,  0.9900,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.8700,  0.9600,  

In [9]:
cnn_small_local_uap_path_withEntropy = "local_uap/local_uap_delta_CNN_small_withEntropy.pt"
train_local_uap_with_entropy_maximization(
    cnn_small,
    cnn_small_local_uap_path_withEntropy,
    train_dataset=small_train_dataset,
    epsilon=epsilon,
    device=device
)

Epoch 1/5: 100%|██████████| 79/79 [00:00<00:00, 85.38it/s]


[Epoch 1] Entropy Loss: 82.5061


Epoch 2/5: 100%|██████████| 79/79 [00:01<00:00, 65.79it/s]


[Epoch 2] Entropy Loss: 105.7882


Epoch 3/5: 100%|██████████| 79/79 [00:00<00:00, 87.31it/s]


...

Epoch 4/5: 100%|██████████| 79/79 [00:00<00:00, 107.63it/s]


[Epoch 4] Entropy Loss: 117.4781


Epoch 5/5: 100%|██████████| 79/79 [00:00<00:00, 107.60it/s]

[Epoch 5] Entropy Loss: 118.0218
Entropy-maximizing Local UAP saved to local_uap/local_uap_delta_CNN_small_withEntropy.pt


tensor([[[[ 0.5100,  0.1900,  0.7300,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.1100,  0.7100, -0.9900,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.5500, -0.4900,  0.9800,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.7000,  0.3700,  0.9500,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.7300,  0.3300, -0.3300,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.4500,  0.7300,  0.9700,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.4100, -1.0000, -1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.2100, -0.9600, -

In [10]:
cnn_local_uap_path_withEntropy = "local_uap/local_uap_delta_CNN_withEntropy.pt"
train_local_uap_with_entropy_maximization(
    cnn,
    cnn_local_uap_path_withEntropy,
    train_dataset=train_dataset,
    epsilon=epsilon,
    device=device
)

Epoch 1/5: 100%|██████████| 391/391 [00:03<00:00, 101.72it/s]


[Epoch 1] Entropy Loss: 629.1805


Epoch 2/5: 100%|██████████| 391/391 [00:04<00:00, 94.83it/s] 


[Epoch 2] Entropy Loss: 672.7418


Epoch 3/5: 100%|██████████| 391/391 [00:03<00:00, 103.19it/s]


...

Epoch 4/5: 100%|██████████| 391/391 [00:03<00:00, 105.19it/s]


[Epoch 4] Entropy Loss: 683.3749


Epoch 5/5: 100%|██████████| 391/391 [00:03<00:00, 102.50it/s]

[Epoch 5] Entropy Loss: 685.4275
Entropy-maximizing Local UAP saved to local_uap/local_uap_delta_CNN_withEntropy.pt


tensor([[[[-0.9600, -0.6300,  0.3700,  ...,  0.0000,  0.0000,  0.0000],
          [-0.5600, -0.9900, -0.1100,  ...,  0.0000,  0.0000,  0.0000],
          [-0.9200, -1.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-0.9900,  0.9800,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.4400, -0.8500,  0.7800,  ...,  0.0000,  0.0000,  0.0000],
          [-1.0000,  1.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[-1.0000, -0.2000, -0.9900,  ...,  0.0000,  0.0000,  0.0000],
          [-0.9400, -0.9600, -